In [1]:
import cv2
import numpy as np
import os
import json
import math

In [2]:
path = r"D:\MAMAA2\Aging\Code\ImageCpy\check"
config_path = 'config/cam1.json'

scale = 0.5
kernel = np.ones((7, 7), np.uint8)
with open(config_path) as f:
    data = json.load(f)

In [3]:
file_list = [path + "\\" + i for i in os.listdir(path)]
imgsize = cv2.imread(file_list[0]).shape

In [4]:
roi_img = np.zeros(imgsize[:-1], np.uint8)
roi_img = cv2.fillPoly(roi_img, [np.array(data['roi'])], 255)

In [5]:
img_obj = np.zeros(imgsize[:-1], np.int)
detect_type = data['detect_type']
img_judge_list = [np.zeros(imgsize[:-1], np.int) for i in range(len(detect_type))]


In [8]:
list_gray = []

area_spec = data['area_spec']
width_spec = data['width_spec']
height_spec = data['height_spec']
obj_gray_spec = data['obj_gray_spec']

for img_path in file_list:
    img = cv2.imread(img_path)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.blur(gray, (7, 7), 2)

    #gray_blur = cv2.equalizeHist(gray_blur)
    gray_blur = cv2.bitwise_and(gray_blur, gray_blur, mask=roi_img)

    #取得當下ROI區域的灰階平均值
    gray_value = gray_blur[roi_img == 255].mean()
    if (gray_value < obj_gray_spec[0] or gray_value > obj_gray_spec[1]):
        continue
    
    list_gray.append(gray_value)

    #計算檢測type區域
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    for i, tp in enumerate(detect_type):
        hsv_lower = np.array(data[tp]['HSV_LOWER'])
        hsv_upper = np.array(data[tp]['HSV_UPPER'])
        mask = cv2.inRange(img_hsv, hsv_lower, hsv_upper)
        img_judge_list[i][mask != 0] += 1

    #切割當下物件區域
    ret2, th_otsu = cv2.threshold(gray_blur, 0, 255,
                                  cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    th_otsu = cv2.erode(th_otsu, kernel, iterations=3)
    th_otsu = cv2.dilate(th_otsu, kernel, iterations=3)
    contours, hierarchy = cv2.findContours(th_otsu, cv2.RETR_CCOMP,
                                           cv2.CHAIN_APPROX_NONE)
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        if (area < area_spec[0]) or (area > area_spec[1]) or (w < width_spec[0]) or (
        w > width_spec[1]) or (h < height_spec[0]) or (h > height_spec[1]):
            th_otsu = cv2.drawContours(th_otsu, [c], -1, 0, thickness=cv2.FILLED)

    #cv2.imshow('otsu', cv2.resize(th_otsu, (int(img.shape[1] * scale),int(img.shape[0] * scale)), interpolation=cv2.INTER_AREA))
    #cv2.waitKey(0)
    img_obj[th_otsu != 0] += 1

cv2.destroyAllWindows()

In [9]:
# 找所有物件的候選區域
obj_gray_spec = data['obj_gray_spec']
obj_occur_ratio = data['obj_occur_ratio']
valid_count = len(list_gray)
obj_detect_th = valid_count * obj_occur_ratio
img_sheet = np.zeros(imgsize[:-1], np.uint8)
img_sheet[img_obj >= obj_detect_th] = 255

In [10]:
# 計算該出現物件的左上座標
start_pt = data['roi'][0]
end_pt = data['roi'][2]
obj_count = data['obj_count']
obj_pos = []
for i in range(obj_count[0]):
    for j in range(obj_count[1]):
        newx = start_pt[0] + (end_pt[0] - start_pt[0]) / obj_count[1] * j
        newy = start_pt[1] + (end_pt[1] - start_pt[1]) / obj_count[0] * i
        obj_pos.append([newx, newy, -1, -1, -1])
        #[pox_x,pox_y, distance, contours_idx, contours_area]

In [11]:
# 分析每個候選區域與預定出現座標，留下距離每個左上座標最近的候選物件
contours, hierarchy = cv2.findContours(img_sheet, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
for i, c in enumerate(contours):
    x, y, w, h = cv2.boundingRect(c)
    
    min_d = 99999
    obj_pos_idx = -1
    for j in range(len(obj_pos)):
        d = math.hypot(obj_pos[j][0] - x, obj_pos[j][1] - y)
        if d < min_d:
            min_d = d
            obj_pos_idx = j
    if(obj_pos_idx != -1):
        if (min_d < obj_pos[obj_pos_idx][2]) or obj_pos[obj_pos_idx][2] == -1:
            obj_pos[obj_pos_idx][2] = min_d
            obj_pos[obj_pos_idx][3] = i
            



In [16]:
# 畫結果模板圖
img_result_temp = np.zeros(imgsize, np.uint8)
for i, obj in enumerate(obj_pos):
    if obj[2] != -1:
        img_result_temp = cv2.drawContours(img_result_temp, [contours[obj[3]]], -1, (255,255,255), thickness=cv2.FILLED)
        obj_pos[i][4] = int(cv2.contourArea(contours[obj[3]]))

In [18]:
# 畫物件結果圖
img_result_obj = img_result_temp.copy()
for obj in obj_pos:
    txt = ""
    color = (0, 255, 0)
    if obj[2] != -1:
        txt = str(obj[4])
        color = (0, 255, 0)
    else:
        txt = "NG(Nofound)"
        color = (0, 0, 255)
    img_result_obj = cv2.putText(img_result_obj, txt, (int(obj[0]), int(obj[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
img_result_obj = cv2.putText(img_result_obj, "Object Result", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

In [55]:
# 畫檢測type圖
img_result_type = img_result_temp.copy()
img_type = np.zeros(imgsize[:-1], np.uint8)
img_type[img_judge_list[0] > valid_count * 0.6] = 255
for i, obj in enumerate(obj_pos):
    img_temp = np.zeros(imgsize[:-1], np.uint8)
    if obj[2] != -1:
        img_temp = cv2.drawContours(img_temp, [contours[obj[3]]], -1, 255, thickness=cv2.FILLED)
    break

In [30]:
#obj_pos

[[132.0, 402.0, 82.29823813423954, 10, 34690],
 [498.5, 402.0, 52.02163011671203, 9, 45806],
 [132.0, 653.5, 87.3226774669673, 7, 40899],
 [498.5, 653.5, 26.61766330841233, 6, 52228],
 [132.0, 905.0, -1, -1, -1],
 [498.5, 905.0, 11.672617529928752, 4, 47449],
 [132.0, 1156.5, 160.22562217073772, 2, 36162],
 [498.5, 1156.5, 107.23105893350116, 1, 24293]]

In [ ]:
#np.count_nonzero(d>binary_th)